In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, RocCurveDisplay

from tqdm import tqdm

import optuna

In [2]:
data = np.load('processed_ntuples\chunk_data.npy')
np.shape(data)

(529663, 33)

In [3]:
train, val = train_test_split(data, random_state=137) # setting the seed for reproducibility; the default train/test split is 0.75/0.25
print('lenght of training set', len(train), '\nlenght of validation set', len(val), '\nwidth of the sets: # of features + 1 flag (1-LL/0-TX)', len(train[0]))

lenght of training set 397247 
lenght of validation set 132416 
width of the sets: # of features + 1 flag (1-LL/0-TX) 33


In [4]:
# strip the last feature (e.g. the flag) from the sets

# copy in dedicated arrays
flags_train = train[:, -1]
flags_val = val[:, -1]
# delete from the sets
train = train[:, :-1]
val = val[:, :-1]
# width check
print(len(train[0]), len(val[0]))

32 32


In [5]:
# tensorize the data, so that pytorch doesn't whine
train = torch.tensor(train, dtype=torch.float32)
val = torch.tensor(val, dtype=torch.float32)
flags_train = torch.tensor(flags_train, dtype=torch.float32)
flags_val = torch.tensor(flags_val, dtype=torch.float32)

In [6]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0.01):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_score = None

    def early_stop(self, validation_loss):
        score = -validation_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.best_score= validation_loss
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.best_score = score
            self.counter = 0
        return False

In [7]:
# 1. Define an objective function to be maximized.
def objective(trial):

    n_epochs = 50
    batch_size = 1000 # size of each batch
    batch_start = torch.arange(0, len(train), batch_size)

    loss_function = nn.BCELoss() # CrossEntropyLoss but for just one class

    # Early stopping
    early_stopping = EarlyStopping(patience=5, delta=0.01)

    # 2. Suggest values of the hyperparameters using a trial object.
    n_layers = trial.suggest_int('n_layers', 2, 4)
    layers = []

    in_features = len(train[0])
    for i in range(n_layers):
        out_features = trial.suggest_int(f'n_units_l{i}', 40, 100)
        layers.append(torch.nn.Linear(in_features, out_features))
        layers.append(nn.BatchNorm1d(out_features))
        layers.append(torch.nn.ReLU())
        layers.append(torch.nn.Dropout(0.25))
        in_features = out_features
    layers.append(torch.nn.Linear(in_features, 1))
    layers.append(nn.Sigmoid())
    model = torch.nn.Sequential(*layers).to(torch.device('cpu'))

    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(n_epochs):
        model.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                train_batch = train[start:start+batch_size]
                flags_train_batch = flags_train[start:start+batch_size]
                # forward pass
                outputs = model(train_batch)
                loss = loss_function(outputs, flags_train_batch.unsqueeze(0).T)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (outputs.round() == flags_train_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )

        model.eval()
        with torch.no_grad():
            val_outputs = model(val)
            val_loss = loss_function(val_outputs, flags_val.unsqueeze(0).T)
            accuracy = float((val_outputs.round().T == flags_val).float().mean())

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
        # Early Stopping
        if early_stopping.early_stop(val_loss):
            print(f'Early stopped at epoch: {epoch}')
            break

    early_stopping = EarlyStopping(patience=5, delta=0.01)

    return accuracy


# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

[I 2025-03-06 21:37:56,424] A new study created in memory with name: no-name-15f0432c-2dc6-49f4-948a-dc8a4146524d
[I 2025-03-06 21:38:35,016] Trial 0 finished with value: 0.7210608720779419 and parameters: {'n_layers': 3, 'n_units_l0': 43, 'n_units_l1': 70, 'n_units_l2': 46, 'lr': 0.0004086088999931899}. Best is trial 0 with value: 0.7210608720779419.


Early stopped at epoch: 5


[I 2025-03-06 21:39:01,780] Trial 1 finished with value: 0.7520465850830078 and parameters: {'n_layers': 2, 'n_units_l0': 47, 'n_units_l1': 70, 'lr': 0.03333684738573851}. Best is trial 1 with value: 0.7520465850830078.


Early stopped at epoch: 5


[I 2025-03-06 21:39:45,179] Trial 2 finished with value: 0.7554298639297485 and parameters: {'n_layers': 3, 'n_units_l0': 61, 'n_units_l1': 52, 'n_units_l2': 46, 'lr': 0.0016797858211212325}. Best is trial 2 with value: 0.7554298639297485.


...

[I 2025-03-07 00:36:45,521] Trial 379 pruned. 
[I 2025-03-07 00:36:51,978] Trial 380 pruned. 
[I 2025-03-07 00:37:23,692] Trial 381 pruned. 
[I 2025-03-07 00:38:05,000] Trial 382 pruned. 
[I 2025-03-07 00:38:19,251] Trial 383 pruned. 
[I 2025-03-07 00:38:25,342] Trial 384 pruned. 
[I 2025-03-07 00:39:06,708] Trial 385 pruned. 
[I 2025-03-07 00:39:29,825] Trial 386 pruned. 
[I 2025-03-07 00:40:01,156] Trial 387 pruned. 
[I 2025-03-07 00:40:24,669] Trial 388 pruned. 
[I 2025-03-07 00:40:37,946] Trial 389 pruned. 
[I 2025-03-07 00:41:14,631] Trial 390 pruned. 
[I 2025-03-07 00:41:37,780] Trial 391 pruned. 
[I 2025-03-07 00:42:42,351] Trial 392 finished with value: 0.8685657382011414 and parameters: {'n_layers': 2, 'n_units_l0': 97, 'n_units_l1': 90, 'lr': 0.014297912006794821}. Best is trial 302 with value: 0.8724852204322815.


Early stopped at epoch: 13


[I 2025-03-07 00:42:50,899] Trial 393 pruned. 
[I 2025-03-07 00:43:00,512] Trial 394 pruned. 
[I 2025-03-07 00:43:04,896] Trial 395 pruned. 
[I 2025-03-07 00:43:19,191] Trial 396 pruned. 
[I 2025-03-07 00:43:32,849] Trial 397 pruned. 
[I 2025-03-07 00:44:10,032] Trial 398 pruned. 
[I 2025-03-07 00:44:42,260] Trial 399 pruned. 
[I 2025-03-07 00:44:55,359] Trial 400 pruned. 
[I 2025-03-07 00:45:36,521] Trial 401 pruned. 
[I 2025-03-07 00:46:09,784] Trial 402 pruned. 
[I 2025-03-07 00:46:18,647] Trial 403 pruned. 
[I 2025-03-07 00:46:50,196] Trial 404 pruned. 
[I 2025-03-07 00:46:55,097] Trial 405 pruned. 
[I 2025-03-07 00:46:59,944] Trial 406 pruned. 
[I 2025-03-07 00:47:32,150] Trial 407 pruned. 
[I 2025-03-07 00:48:03,130] Trial 408 pruned. 
[I 2025-03-07 00:49:05,630] Trial 409 finished with value: 0.8677727580070496 and parameters: {'n_layers': 2, 'n_units_l0': 99, 'n_units_l1': 97, 'lr': 0.01463290870196419}. Best is trial 302 with value: 0.8724852204322815.


Early stopped at epoch: 12


[I 2025-03-07 00:49:20,031] Trial 410 pruned. 
[I 2025-03-07 00:50:04,003] Trial 411 pruned. 
[I 2025-03-07 00:50:28,533] Trial 412 pruned. 
[I 2025-03-07 00:50:35,752] Trial 413 pruned. 
[I 2025-03-07 00:50:40,398] Trial 414 pruned. 
[I 2025-03-07 00:51:33,634] Trial 415 pruned. 
[I 2025-03-07 00:51:47,836] Trial 416 pruned. 
[I 2025-03-07 00:52:35,721] Trial 417 pruned. 
[I 2025-03-07 00:53:07,713] Trial 418 pruned. 
[I 2025-03-07 00:53:43,459] Trial 419 pruned. 
[I 2025-03-07 00:54:28,029] Trial 420 pruned. 
[I 2025-03-07 00:54:32,628] Trial 421 pruned. 
[I 2025-03-07 00:55:18,351] Trial 422 pruned. 
[I 2025-03-07 00:55:55,887] Trial 423 pruned. 
[I 2025-03-07 00:56:49,615] Trial 424 pruned. 
[I 2025-03-07 00:57:25,531] Trial 425 pruned. 
[I 2025-03-07 00:57:49,185] Trial 426 pruned. 
[I 2025-03-07 00:58:36,359] Trial 427 pruned. 
[I 2025-03-07 00:59:14,733] Trial 428 pruned. 
[I 2025-03-07 00:59:24,350] Trial 429 pruned. 
[I 2025-03-07 00:59:29,221] Trial 430 pruned. 
[I 2025-03-07

In [8]:
print("Best trial:")
b_trial = study.best_trial
print("  Value: ", b_trial.value)

print("  Params: ")
for key, value in b_trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  0.8724852204322815
  Params: 
    n_layers: 2
    n_units_l0: 98
    n_units_l1: 84
    lr: 0.008655714654422043


In [10]:
file = open('optuna_res.txt', 'w')

for key, value in b_trial.params.items():
    print("    {}: {}".format(key, value))
    file.write("    {}: {}\n".format(key, value))

file.close()

    n_layers: 2
    n_units_l0: 98
    n_units_l1: 84
    lr: 0.008655714654422043


In [9]:
#